In [192]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd



In [193]:
#load data
data = pd.read_csv('Symptom2Disease.csv')

#split data into train and test
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)

print(len(train_data))
print(f"{len(test_data)}, {test_data.shape})")

for i,x in enumerate(test_data):
    if(i == 10):
        break
    print(x)
    


960
240, (240, 3))
Unnamed: 0
label
text


In [194]:
#connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [195]:
#train data
train_labels = train_data['label'].values
train_text = train_data['text'].values

#test data
test_labels = test_data['label'].values
test_text = test_data['text'].values

In [196]:
#tokenize data
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#tokenize and convert each sentence into tensors
input_ids = []
for sent in train_text:

    encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
    input_ids.append(torch.tensor(encoded_sent))
    

test_input_ids = []
for sent in test_text:
        encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
        test_input_ids.append(torch.tensor(encoded_sent))

#tokenize and convert each sentence into tensors
label_ids = []
for label in train_labels:
    
    encoded = tokenizer.encode(label,add_special_tokens = True)
    label_ids.append(torch.tensor(encoded))

test_label_ids = []
for label in test_labels:
    encoded = tokenizer.encode(label,add_special_tokens = True)
    test_label_ids.append(torch.tensor(encoded))


print('Original: ', train_text[0])
print('Token IDs:', input_ids[0][0:5])



Original:  I have seen rashes on my arms and neck and it itches if I scratch them. I've also had a high fever for a few days. I have no idea what is causing it. The itching is causing me a lot of discomforts.
Token IDs: tensor([  101,  1045,  2031,  2464, 23438])


In [197]:
from torch.nn.utils.rnn import pad_sequence

#pad the tensors to make them of equal length
padded_input_ids = pad_sequence(input_ids, batch_first=True) #78 size
testing = pad_sequence(test_input_ids, batch_first=True)  #68 size

#pad them so they are the same size
padded_test_input_ids = torch.nn.functional.pad(testing, (0, 78 - testing.size(1)), value=0)

print(padded_input_ids.shape)
print(padded_test_input_ids.shape)




#pad the tensors to make them of equal length
padded_labels = pad_sequence(label_ids, batch_first=True)
padded_test_labels = pad_sequence(test_label_ids, batch_first=True, padding_value = 0)





torch.Size([960, 78])
torch.Size([240, 78])


In [198]:



input_size = padded_input_ids.shape[1]
hidden_size = 50
output_size = padded_labels.shape[1]

input_size1 = padded_test_input_ids.shape[1]
hidden_size = 50
output_size1 = padded_test_labels.shape[1]

print(padded_input_ids.shape)

print(f"input_size: {input_size}, output_size: {output_size}, hidden_size: {hidden_size}")
print(f"input_size1: {input_size1}, output_size1: {output_size1}, hidden_size: {hidden_size}")


torch.Size([960, 78])
input_size: 78, output_size: 12, hidden_size: 50
input_size1: 78, output_size1: 12, hidden_size: 50


In [199]:
#create a simple fnn model that takes in the input size
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out
    
#rnn model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out
    
#LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out
    def predict(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out
    
    




In [200]:
#define hyperparameters
learning_rate = 0.001
epochs = 10
batch_size = 1

print(input_size)
seq_len_train = padded_input_ids.size(0)
seq_len_test = padded_test_input_ids.size(0)
seq_len_train1 = padded_labels.size(0)
seq_len_test1 = padded_test_labels.size(0)



print(seq_len_train)


#seq len == input_size

78
960


In [207]:
model_num = 1
model = None
if(model_num == 0):
    model = FNN(input_size, hidden_size, output_size)

elif(model_num == 1):
    #Reshape for RNN
    padded_input_ids = padded_input_ids.reshape(1, seq_len_train, input_size)
    padded_test_input_ids = padded_test_input_ids.reshape(1, seq_len_test, input_size1)
    padded_labels = padded_labels.reshape(1, seq_len_train1, output_size)
    padded_test_labels = padded_test_labels.reshape(1, seq_len_test1, output_size1)
    
    print(padded_input_ids.shape)
    print(padded_test_input_ids.shape)
    print(padded_labels.shape)
    print(padded_test_labels.shape)
    
    model = RNN(input_size, hidden_size, output_size)


model.to(device)

torch.Size([1, 960, 78])
torch.Size([1, 240, 78])
torch.Size([1, 960, 12])
torch.Size([1, 240, 12])


RNN(
  (rnn): RNN(78, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=12, bias=True)
  (softmax): Softmax(dim=0)
)

In [202]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [226]:
model.train()
#train the model
for epoch in range(epochs):
    for i in range(0, seq_len_train):
        inputs = padded_input_ids.to(device)
        labels = padded_labels.to(device)
        if(i == 0):
            print(inputs[0])
            print(labels[0])
        
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{seq_len_train}], Loss: {loss.item():.4f}')

print('Finished Training')

tensor([[  101,  1045,  2031,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0],
        [  101,  2026, 10063,  ...,     0,     0,     0],
        ...,
        [  101,  2067,  3255,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0],
        [  101,  1045,  2031,  ...,     0,     0,     0]], device='cuda:0')
tensor([[  101,  7975, 13433,  ...,     0,     0,     0],
        [  101, 28079,  8985,  ...,     0,     0,     0],
        [  101,  8827, 11069,  ...,     0,     0,     0],
        ...,
        [  101, 28711, 11867,  ...,     0,     0,     0],
        [  101,  8827, 11069,  ...,     0,     0,     0],
        [  101, 28711, 11867,  ...,     0,     0,     0]], device='cuda:0')


c:\Users\bobth\Downloads\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 960, 12])) that is different to the input size (torch.Size([1, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Step [10/960], Loss: 61041380.0000
Epoch [1/10], Step [20/960], Loss: 61041380.0000
Epoch [1/10], Step [30/960], Loss: 61041380.0000
Epoch [1/10], Step [40/960], Loss: 61041380.0000
Epoch [1/10], Step [50/960], Loss: 61041380.0000
Epoch [1/10], Step [60/960], Loss: 61041380.0000
Epoch [1/10], Step [70/960], Loss: 61041380.0000
Epoch [1/10], Step [80/960], Loss: 61041380.0000
Epoch [1/10], Step [90/960], Loss: 61041380.0000
Epoch [1/10], Step [100/960], Loss: 61041380.0000
Epoch [1/10], Step [110/960], Loss: 61041380.0000
Epoch [1/10], Step [120/960], Loss: 61041380.0000
Epoch [1/10], Step [130/960], Loss: 61041380.0000
Epoch [1/10], Step [140/960], Loss: 61041380.0000
Epoch [1/10], Step [150/960], Loss: 61041380.0000
Epoch [1/10], Step [160/960], Loss: 61041380.0000
Epoch [1/10], Step [170/960], Loss: 61041380.0000
Epoch [1/10], Step [180/960], Loss: 61041380.0000
Epoch [1/10], Step [190/960], Loss: 61041380.0000
Epoch [1/10], Step [200/960], Loss: 61041380.0000
Epoch [1/

KeyboardInterrupt: 

In [217]:
model.eval()
#test the model
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(0, seq_len_test):
        inputs = padded_test_input_ids.to(device)
        labels = padded_test_labels.to(device)
        
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        _, actual = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == actual).sum().item()
        
    print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')



Accuracy of the network on the 240 test images: 100.0 %


In [225]:
#f1score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

with torch.no_grad():
    correct = 0
    total = 0
    y_pred = []
    y_true = []
    for i in range(0, seq_len_test):
        inputs = padded_test_input_ids.to(device)
        labels = padded_test_labels.to(device)
        
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        _, actual = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == actual).sum().item()
        y_pred.append(predicted)
        y_true.append(actual)
    
    #[y_pred, y_true] = [torch.cat(y_pred, dim=0).cpu().numpy(), torch.cat(y_true, dim=0).cpu().numpy()]

    print(y_pred)
    print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')
    print(f'f1 score: {f1_score(y_true, y_pred, average="macro")}')
    print(f'precision score: {precision_score(y_true, y_pred, average="macro")}')
    print(f'recall score: {recall_score(y_true, y_pred, average="macro")}')

Accuracy of the network on the 240 test images: 100.0 %


ValueError: Classification metrics can't handle a mix of multiclass-multioutput and binary targets